This notebook guides through the process of semi-manually segmenting the background of the Transmission channel. The obtained mask sparsifies the image data significantly and is needed for computing astar distances through the microstructure. First load the Transmission channel into napari. Perform prewitt edge detection, gaussian denoising, binarize using otsu, run binary closing. Then comes the manual part where one needs to close all the edges. Once the has been done, use flood to segement the PDMS structure channels. Two masks are saved, one with the explant wel included, one with them excluded.

In [3]:
import napari
from tifffile import imread
import numpy as np

from skimage import filters
from skimage import morphology
from skimage.filters import threshold_otsu
from skimage.segmentation import flood


In [4]:
%gui qt

inp_path = '/run/media/loaloa/lbb_ssd/timelapse1_40_min_processed/'
outp_path = '../tl140_outputdata/'
tranl_file = inp_path+'G001_grey_compr.deflate.tif'
transl_ch = imread(tranl_file)[0]

In [5]:
with napari.gui_qt():
    viewer = napari.Viewer()

    viewer.add_image(transl_ch)
    viewer.layers['transl_ch'].blending='additive'
    viewer.layers['transl_ch'].contrast_limits = [.007*2**16, .0191*2**16]
    viewer.layers['transl_ch'].opacity = .7
    viewer.layers['transl_ch'].gamma = .72

In [6]:
# edge detection
prewitt = filters.prewitt(transl_ch)
viewer.add_image(prewitt)

<Image layer 'prewitt' at 0x7fa42b2ea340>

In [7]:
# perform guassian smoothing on edge filtered image
prewitt_gaussian = filters.gaussian(prewitt, sigma=1)
viewer.add_image(prewitt_gaussian)

<Image layer 'prewitt_gaussian' at 0x7fa444050d30>

In [8]:
# binarize image
prewitt_bin = prewitt > threshold_otsu(prewitt_gaussian)
viewer.add_image(prewitt_bin)

<Image layer 'prewitt_bin' at 0x7fa438581f40>

In [11]:
# Run this, then use the bucket tool to fill everything. 
# Play with sigma and solem diameter if things look bad.
# delete parts ofthe image where channels almost touch each other
diameter = 4
selem = morphology.selem.square(diameter)
prewitt_bin_closed = morphology.binary_closing(prewitt_bin, selem)
viewer.add_labels(prewitt_bin_closed)

<Labels layer 'prewitt_bin_closed [1]' at 0x7fa42867bc70>

In [12]:
# # If the data was already filled, load the mask like that
# prewitt_bin_closed = np.load(f"{outp_path}structure_mask.npy")
# viewer.add_labels(prewitt_bin_closed)

# Or save below:
np.save(f"{outp_path}structure_mask.npy")

<Labels layer 'prewitt_bin_closed [1]' at 0x7fa4283b12b0>

In [14]:
# Flood (1000,2700) is the start point
prewitt_bin_closed_flooded = flood(prewitt_bin_closed, (1000,2700))
# fill wells afterwards and drawy cutoff segmentation
viewer.add_labels(prewitt_bin_closed_flooded, name='wells_incl')
viewer.add_labels(np.zeros_like(prewitt), name='y_cutoff')

<Labels layer 'y_cutoff' at 0x7fa41ff01820>

In [19]:
wells_incl_mask = viewer.layers['wells_incl'].data
wells_excl_mask = prewitt_bin_closed_flooded.copy()
wells_excl_mask[viewer.layers['y_cutoff'].data.astype(bool)] = 0

selem = morphology.selem.diamond(8)
wells_incl_mask_clean = filters.median(wells_incl_mask, selem)
wells_excl_mask_clean = filters.median(wells_excl_mask, selem)
# final fixing
viewer.add_labels(wells_incl_mask_clean, name='final_incl')
viewer.add_labels(wells_excl_mask_clean, name='final_excl')

(2920, 5764)
bool
(2920, 5764)
bool


In [21]:
np.save(f"{outp_path}mask_wells_incl.npy", viewer.layers['final_excl'].data)
np.save(f"{outp_path}mask_wells_excl.npy", viewer.layers['final_excl'].data)